<a href="https://colab.research.google.com/github/yashinaniya/ML-exercises/blob/main/Predicting_happy_n_sad_images_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [ ]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

--2021-01-02 17:53:45--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.5.240, 172.217.164.176, 172.217.15.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.5.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2021-01-02 17:53:45 (107 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [ ]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential()
# Step 1 - Convolution
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[150, 150, 3]))

# Step 2 - Pooling
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
model.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
model.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer = RMSprop(lr=0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   )

train_generator = train_datagen.flow_from_directory("/tmp/h-or-s",
                                                 target_size = (150, 150),
                                                 batch_size = 10,
                                                 class_mode = 'binary' )

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [ ]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit( train_generator, 
                    steps_per_epoch = 8,
                    epochs = 20,
                    verbose = 1,
                    callbacks = [callbacks]
                   )
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
8/8 [==============================] - 7s 19ms/step - loss: 3.4873 - accuracy: 0.5139
Epoch 2/20
8/8 [==============================] - 0s 20ms/step - loss: 0.4211 - accuracy: 0.8259
Epoch 3/20
8/8 [==============================] - 0s 20ms/step - loss: 0.2463 - accuracy: 0.9284
Epoch 4/20
8/8 [==============================] - 0s 20ms/step - loss: 0.1858 - accuracy: 0.9237
Epoch 5/20
8/8 [==============================] - 0s 20ms/step - loss: 0.0635 - accuracy: 0.9795
Epoch 6/20
8/8 [==============================] - 0s 20ms/step - loss: 0.0755 - accuracy: 0.9752
Epoch 7/20
8/8 [==============================] - 0s 19ms/step - loss: 0.1094 - accuracy: 0.9287
Epoch 8/20
8/8 [==============================] - 0s 20ms/step - loss: 0.0691 - accuracy: 0.9613
Epoch 9/20
8/8 [==============================] - 0s 19ms/step - loss: 0.1234 - accuracy: 0.9368
Epoch 10/20
8/8 [==============================] - 0s 20ms/step - loss: 0.0099 - accuracy: 1.0000

Reached 99% accuracy so canc